## Beam deflection with double integration

In [ ]:
%matplotlib widget 
# Documentation needed for ipyml - notifications on push from github
from bmcs_utils.api import InteractiveModel, IPWInteract
import traits.api as tr
import matplotlib.pylab as plt
from matplotlib.path import Path
import matplotlib.patches as mpatches
from matplotlib.patches import PathPatch
import numpy as np
import sympy as sp
from scipy.integrate import cumtrapz
sp.init_printing()


In [ ]:
class BeamBC(tr.HasTraits):
    phi0_list = tr.List
    w0_list = tr.List
    
class BeamRC3Pt(BeamBC):
    pass

In [ ]:
x, M, L, P= sp.symbols('x, M, L, P')
PtoM = {P : M*4/L}
# M_ = 2*M/L - 4*M/L*(x-L/2)
M_ = sp.Piecewise(
    (0,x<0),
    (2*M*x/L, x <= L/2),
    ((2*M*x/L - (4*M/L)*(x-L/2)), x<=L),
    (0, True)
            )
M_

In [ ]:
get_M = sp.lambdify((x,M,L), M_)

In [ ]:
phi_ = sp.integrate(M_,(x))
phi_

In [ ]:
# phi_ = sp.integrate(M_,x)
phi_ = sp.Piecewise((M*x**2/L - P*L**2/16, x <= L/2),
    (P*x**2/4 - P*L**2/16 - P/2*(x-L/2)**2, x > L/2))
         
phi_

In [ ]:
phi_ = phi_.subs(PtoM)
phi_

In [ ]:
get_phi = sp.lambdify((x,M,L), phi_)

In [ ]:
w_ = sp.integrate(phi_,x)
w_

In [ ]:
get_w = sp.lambdify((x,M,L), w_)

In [ ]:
from bmcs_beam.models.moment_curvature.moment_curvature_ import MomentCurvature
mc = MomentCurvature()


In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
class Beam3PtBending(InteractiveModel):
    '''
    Todo: 1- how to have tr.int
          2- divide it to more classes
          3- moving point load, line load, load combinations
    '''
    name = 'Analysis'
    
#     mc_function = tr.Instance(MomentCurvature)
#     mc_function = MomentCurvature(M)
#     def _mc_function_default(self):
#         return MomentCurvature()
    
    # Reinforcement
    E_carbon = tr.Int(200000, param=True, latex='E_r \mathrm{MPa}', minmax=(200000,300000)) 
    width = tr.Float(10, param=True, latex='rov_w \mathrm{mm}', minmax=(10,450)) 
    thickness = tr.Float(1, param=True, latex='rov_t \mathrm{mm}', minmax=(1,100))
    spacing = tr.Float(1, param=True, latex='ro_s \mathrm{mm}', minmax=(1,100))
    n_layers = tr.Int(1, param=True, latex='n_l \mathrm{-}', minmax=(1,100))
    A_roving = tr.Float(1, param=True, latex='A_r \mathrm{mm2}', minmax=(1,100))
    
    #Concerte cross section
    L = tr.Int(500, param=True, latex='L \mathrm{mm}', minmax=(10,10000))
    H = tr.Int(10, param=True, latex='H \mathrm{mm}', minmax=(10,500))
    B = tr.Int(10, param=True, latex='B \mathrm{mm}', minmax=(10,500))
    E_con = tr.Int(14000, param=True, latex='E \mathrm{MPa}', minmax=(14000,41000))
    F = tr.Float(1000, param=True, latex='F \mathrm{N}', minmax=(10,10000))
    n_x = tr.Int(100, param=True, latex='n_x \mathrm{-}', minmax=(1,1000))
    G_adj = tr.Float(0.02, param=True, latex='G_adj \mathrm{-}', minmax=(1e-3,1e-1))
    param_names = ['L', 'H', 'B', 'E_con' ,'F', 'E_carbon', 'width','thickness', 'spacing', 'n_layers', 'A_roving','G_adj']
    
    x = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_x(self):
        return np.linspace(0,self.L,self.n_x)
    
    E_comp = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_E_comp(self):
        A_composite = self.B * self.H
        n_rovings = self.width / self.spacing 
        A_layer = n_rovings * self.A_roving 
        A_carbon = self.n_layers * A_layer 
        A_concrete = A_composite - A_carbon 
        E_comp = (self.E_carbon * A_carbon + self.E_con * A_concrete) / (A_composite)
        return E_comp   

    def get_M_x(self):
        M = self.F / 2 * self.L / 2
        M_x = get_M(self.x, M, self.L)
        return M_x
    
    def get_kappa_x(self):
        M = self.get_M_x()
#         I = (self.B*self.H**3)/12
        return mc.get_kappa(M)
#         return M / I / self.E_comp
    
    def get_phi_x(self):
        kappa_x = self.get_kappa_x()
        phi_x = cumtrapz(kappa_x, self.x, initial=0)
        phi_L2 = np.interp(self.L/2, self.x, phi_x)
        phi_x -= phi_L2
        return phi_x
    
    def get_w_x(self):
        phi_x = self.get_phi_x()
        w_x = cumtrapz(phi_x, self.x, initial=0)
        w_x += w_x[0]
        return w_x
    
    def subplots(self, fig):
        return fig.subplots(5, 1)

    def update_plot(self, axes):
        ax1, ax2, ax3, ax4, ax5 = axes
        
#         ax1.axis('equal');
        ax1.fill([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='gray')
        ax1.plot([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='black')
                
        vertices = []
        codes = []

        codes = [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices = [(-self.L * self.G_adj, -self.L * self.G_adj), (0, 0),
                    (self.L * self.G_adj, -self.L * self.G_adj), 
                    (-self.L * self.G_adj, -self.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY] 
        vertices += [(self.L * (1-self.G_adj), -self.L * self.G_adj), (self.L, 0),
                     (self.L * (1 + self.G_adj), -self.L * self.G_adj), 
                     (self.L * (1-self.G_adj), -self.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO] + [Path.CLOSEPOLY] 
        vertices += [(self.L * (1+self.G_adj), -self.L * self.G_adj*1.2),
                     (self.L * (1-self.G_adj), -self.L * self.G_adj*1.2),
                     (self.L * (1-self.G_adj), -self.L * self.G_adj*1.2)]

        vertices = np.array(vertices, float)
        path = Path(vertices, codes)

        pathpatch = PathPatch(path, facecolor='None', edgecolor='green')

        ax1.add_patch(pathpatch)
        ax1.plot((0,self.L*1),(0,0))
#         ax1.set_title('A compound path')
        # ax.set_size(3,3)
        # ax.autoscale_view()

        x_tail = self.L/2
        y_tail = self.F * self.G_adj + self.H
        x_head = self.L/2
        y_head = 0 + self.H
        dy = y_head + x_head/10
        arrow = mpatches.FancyArrowPatch((x_tail, y_tail), (x_head, y_head),
                                         mutation_scale=10)
        ax1.add_patch(arrow)
        ax1.axis('auto');
#         ax1.autoscale(tight=True)
        
        x = self.x
        
        M_x = self.get_M_x()
        ax2.plot(x, -M_x, color='red', label='moment [N.mm]')
        leg = ax2.legend();

        phi_x = self.get_phi_x()
        ax3.plot(x, phi_x, color='green', label='phi []')
        leg = ax3.legend();
        
        w_x = self.get_w_x()
        ax4.plot(x, w_x, color='blue', label='$w$ [mm]')
        leg = ax4.legend();
                
        kappa_x = self.get_kappa_x()
        ax5.plot(x, kappa_x, color='black', label='$kappa$ []')
        leg = ax5.legend();
        
class CrossSectionDesgin(InteractiveModel):

    name = 'Section'
   
    # Reinforcement
    E_carbon = tr.Int(200000, param=True, latex='E_r [MPa]', minmax=(200000,300000)) 
    width = tr.Float(8, param=True, latex='rov_w [mm]', minmax=(8,450)) 
    thickness = tr.Float(1, param=True, latex='rov_t [mm]', minmax=(1,100))
    spacing = tr.Float(1, param=True, latex='ro_s [mm]', minmax=(1,100))
    n_layers = tr.Int(1, param=True, latex='n_l [-]', minmax=(1,100))
    A_roving = tr.Float(1, param=True, latex='A_r [mm2]', minmax=(1,100))
    f_h = tr.Int(5, param=True, latex='f_h [mm]', minmax=(5,500))

    #Concerte cross section
    L = tr.Int(5000, param=True, latex='L [mm]', minmax=(10,10000))
    H = tr.Int(10, param=True, latex='H [mm]', minmax=(10,500))
    B = tr.Int(10, param=True, latex='B [mm]', minmax=(10,500))
    E_con = tr.Int(14000, param=True, latex='E [MPa]', minmax=(14000,41000))
    F = tr.Float(10, param=True, latex='F [N]', minmax=(10,1000))
    n_x = tr.Int(100, param=True, latex='n_x', minmax=(1,1000))
    
    param_names = ['H', 'B', 'E_con', 'E_carbon', 'width','thickness', 'spacing', 'n_layers', 'A_roving','f_h']
    
    def get_comp_E(self):
        A_composite = self.B * self.H
        n_rovings = self.width / self.spacing 
        A_layer = n_rovings * self.A_roving 
        A_carbon = self.n_layers * A_layer 
        A_concrete = A_composite - A_carbon 
        E_comp = (self.E_carbon * A_carbon + self.E_con * A_concrete) / (A_composite)
        return E_comp
      
    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):     
        ax.axis([0, self.B, 0, self.H]);
        ax.axis('equal');
        ax.fill([0,self.B,self.B,0,0], [0,0,self.H,self.H,0],color='gray')
        ax.plot([0,self.B,self.B,0,0], [0,0,self.H,self.H,0],color='black')
        ax.plot([self.B/2 - self.width/2, self.B/2 + self.width/2], 
                [self.f_h,self.f_h],color='Blue',
                linewidth=self.n_layers*self.thickness)       
        ax.annotate('E_composite = {} GPa'.format(np.round(self.get_comp_E()/1000),0),
                xy=(self.B/10,self.f_h*1.1), color='white')
        
        
class BoundaryConditions(InteractiveModel):
    
    name = 'BC'
   
    L = tr.Int(5000, param=True, latex='L [mm]', minmax=(5000,10000))
    H = tr.Int(200, param=True, latex='H \mathrm{mm}', minmax=(10,500))
    F = tr.Float(1000, param=True, latex='F [N]', minmax=(1000,10000))
    n_sup = tr.Int(2, param=True, latex='n_sup', minmax=(2,10))
    G_adj = tr.Float(0.02, param=True, latex='G_adj \mathrm{-}', minmax=(1e-3,1e-1))
    F_pos = tr.Int(2500, param=True, latex='F_pos', minmax=(0,10000))

    param_names = ['L','H','F','n_sup','G_adj','F_pos']


    def Support (self,n):
        
        slef.n = n
        pass 
    
    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):  
        
        ax.fill([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='gray')
        ax.plot([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='black')
                
        vertices = []
        codes = []

        codes = [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices = [(-self.L * self.G_adj, -self.L * self.G_adj), (0, 0),
                    (self.L * self.G_adj, -self.L * self.G_adj), 
                    (-self.L * self.G_adj, -self.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY] 
        vertices += [(self.L * (1-self.G_adj), -self.L * self.G_adj), (self.L, 0),
                     (self.L * (1 + self.G_adj), -self.L * self.G_adj), 
                     (self.L * (1-self.G_adj), -self.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO] + [Path.CLOSEPOLY] 
        vertices += [(self.L * (1+self.G_adj), -self.L * self.G_adj*1.2),
                     (self.L * (1-self.G_adj), -self.L * self.G_adj*1.2),
                     (self.L * (1-self.G_adj), -self.L * self.G_adj*1.2)]

        vertices = np.array(vertices, float)
        path = Path(vertices, codes)

        pathpatch = PathPatch(path, facecolor='None', edgecolor='green')

        ax.add_patch(pathpatch)
        ax.plot((0,self.L*1),(0,0))
#         ax.set_title('A compound path')
        # ax.set_size(3,3)
        # ax.autoscale_view()

        x_tail = self.F_pos
        y_tail = self.L/20 + self.H
        x_head = self.F_pos
        y_head = 0 + self.H
        dy = y_head + x_head/10
        arrow = mpatches.FancyArrowPatch((x_tail, y_tail), (x_head, y_head),
                                         color = 'blue',mutation_scale=self.L/500)
        ax.annotate('{} KN'.format(np.round(self.F/1000),0),
                xy=(x_tail,y_tail), color='black')
        ax.add_patch(arrow)
        ax.axis('equal')
        ax.autoscale(tight=True)

In [ ]:
# bd = Beam3PtBending()
ibd = IPWInteract([BoundaryConditions(),CrossSectionDesgin(),Beam3PtBending()])
ibd.interact()